# UCL Online Retail Dataset

This is a transnational data set which contains all the transactions occurring from December 1st 2010 until December 9th 2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers. Each row represents the transaction that occurs. It includes the product name, quantity, price, and other columns that represents ID.

Source: http://archive.ics.uci.edu/ml/datasets/Online+Retail (Dr Daqing Chen, Director: Public Analytics group. chend '@' lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.)

## Column Descriptions

| Column Name  | Description                                                                 | Data Type                                             |
|--------------|-----------------------------------------------------------------------------|------------------------------------------------------|
| **InvoiceNo** | Invoice number. If this code starts with letter 'C', it indicates a cancellation. | Nominal, a 6-digit integral number uniquely assigned to each transaction |
| **StockCode** | Product (item) code                                                       | Nominal, a 5-digit integral number uniquely assigned to each distinct product |
| **Description** | Product (item) name                                                     | Nominal                                              |
| **Quantity**   | The quantities of each product (item) per transaction                    | Numeric                                              |
| **InvoiceDate** | Invoice Date and time when each transaction was generated               | Numeric, the day and time                            |
| **UnitPrice**  | Unit price, Product price per unit in sterling                           | Numeric                                              |
| **CustomerID** | Customer number, uniquely assigned to each customer                      | Nominal, a 5-digit integral number                  |
| **Country**    | Country name, the name of the country where each customer resides        | Nominal                                              |

## Plan
1. reading data and preprocessing
2. creating Recency, Frequency, Monetary table
3. clustering with K-means algorithm
4. interpreting the result

In [2]:
import sys
import pandas as pd 
import numpy as np 
import plotly.express as px 
import plotly as py 
import matplotlib.pyplot as plt 
import seaborn as sns
from collections import Counter 
from PIL import Image 
import pandas as pd 
from pathlib import Path 
import calendar 
%pip install termcolor 
from termcolor import colored 
from IPython.display import HTML
import warnings 
warnings.filterwarnings('ignore')
py.offline.init_notebook_mode(connected=True)

Note: you may need to restart the kernel to use updated packages.


# Step 1 Reading Data

In [3]:
df=pd.read_excel('./Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
print(f'Number of rows in the dataset is {df.shape[0]}')
print(f'Number of columns in the dataset is {df.shape[1]}')


Number of rows in the dataset is 541909
Number of columns in the dataset is 8


# Step 2 Cleaning Data

$$

$$